In [1]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --no-index --find-links=/kaggle/input/wheels-torch-cu121

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
Looking in links: /kaggle/input/wheels-torch-cu121
Processing /kaggle/input/wheels-torch-cu121/torch-2.5.1+cu121-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/wheels-torch-cu121/torchvision-0.20.1+cu121-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/wheels-torch-cu121/torchaudio-2.5.1+cu121-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/wheels-torch-cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch)
Processing /kaggle/input/wheels-torch-cu121/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch)
Processing /kaggle/input/wheels-torch-cu121/nvidia_cuda_cupti_cu12-1

In [2]:
!pip install peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/pip-eedi

Looking in links: /kaggle/input/pip-eedi
Processing /kaggle/input/pip-eedi/peft-0.12.0-py3-none-any.whl
Processing /kaggle/input/pip-eedi/accelerate-1.0.1-py3-none-any.whl
Processing /kaggle/input/pip-eedi/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [3]:
!pip install -q --no-index --find-links=/kaggle/input/wheels-vllm-0-6-4-post1 vllm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gpustat 1.0.0 requires nvidia-ml-py<=11.495.46,>=11.450.129, but you have nvidia-ml-py 12.560.30 which is incompatible.


In [4]:
!cp -r /kaggle/input/gte-large-en-v15-save /kaggle/working/
!cp -r /kaggle/input/scripts-for-bge-weights/configuration.py /kaggle/working/gte-large-en-v15-save/alibaba_combined_model
!cp -r /kaggle/input/scripts-for-bge-weights/modeling.py /kaggle/working/gte-large-en-v15-save/alibaba_combined_model

In [5]:
import json

# Open the file in read+write mode
with open('/kaggle/working/gte-large-en-v15-save/alibaba_combined_model/config.json', 'r+') as file:
    # Load the existing data
    data = json.load(file)
    
    # Update the data with new 'auto_map' entries
    data['auto_map'] = {
        "AutoConfig": "configuration.NewConfig",
        "AutoModel": "modeling.NewModel",
        "AutoModelForMaskedLM": "modeling.NewForMaskedLM",
        "AutoModelForMultipleChoice": "modeling.NewForMultipleChoice",
        "AutoModelForQuestionAnswering": "modeling.NewForQuestionAnswering",
        "AutoModelForSequenceClassification": "modeling.NewForSequenceClassification",
        "AutoModelForTokenClassification": "modeling.NewForTokenClassification"
    }
    
    # Move the file pointer to the beginning
    file.seek(0)
    
    # Write the updated data back to the file
    json.dump(data, file, indent=4)
    
    # Truncate the file in case the new content is shorter
    file.truncate()

In [6]:
with open('/kaggle/working/gte-large-en-v15-save/alibaba_combined_model/config.json', 'r+') as file:
    # Load the existing data
    data = json.load(file)

In [7]:
# =================================
# Libraries
# =================================
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
%env TOKENIZERS_PARALLELISM=true
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
from pathlib import Path
from cuml.neighbors import NearestNeighbors
import torch.nn.functional as F
import gc
from dataclasses import dataclass
from peft import PeftModel
from transformers import (
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from transformers import BitsAndBytesConfig
from concurrent.futures import ThreadPoolExecutor
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import LoraConfig, get_peft_model
from torch import nn, Tensor
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


env: TOKENIZERS_PARALLELISM=true


In [8]:
# =================================
# Constants
# =================================
DATA_DIR = Path("/kaggle/input/eedi-mining-misconceptions-in-mathematics")
TEST_PATH = DATA_DIR / "test.csv"
MISCONCEPTION_PATH = DATA_DIR / "misconception_mapping.csv"

In [9]:
# =================================
# Settings
# =================================

# =================================
# LLM generate
# =================================
llm_generate_model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm_generate_prompt_template = """You are an expert in mathematics. 
Refer to the examples below to identify and describe the misconception that led to the incorrect answer. 
Example1 
ConstructName: Recognise and use efficient methods for mental multiplication 
SubjectName: Mental Multiplication and Division 
Math problem: Tom and Katie are discussing ways to calculate \\( 21 \\times 12 \\) mentally. Tom does \\( 12 \\times 7 \\) and then multiplies his answer by \\( 3 \\); Katie does \\( 21 \\times 6 \\) and then doubles her answer. Who would get the correct answer? 
Incorrect answer: Only Katie 
Misconception: Does not correctly apply the distributive property of multiplication 

Example2 
ConstructName: Multiply a decimal by an integer 
SubjectName: Mental Multiplication and Division 
Math problem: \\( 9.4 \\times 50= \\) 
Incorrect answer: \\( 4700 \\) 
Misconception: When multiplying a decimal by an integer, ignores decimal point and just multiplies the digits 

ConstructName: {ConstructName} 
SubjectName: {SubjectName}
Math problem: {QuestionText} 
Incorrect answer: {AnswerText} 
Misconception: 
"""

                   
def llm_generate_make_prompt(row):
    """
    Generate a prompt based on the given row and template version.
    """
    messages = [
    {"role": "user", "content": llm_generate_prompt_template.format(
                            ConstructName=row["ConstructName"],
                            SubjectName=row["SubjectName"],
                            QuestionText=row["QuestionText"],
                            AnswerText=row["AnswerText"],
                            )}
    ]
    return messages

In [10]:
# =================================
# retrieval
# =================================
tokenizer_path="/kaggle/input/baai-bge-large-en-v15-save/baai-bge-large-en-v15"
baai_bge_large_path = "/kaggle/input/baai-bge-large-en-v15-save/baai-bge-large-en-v15"
baai_bge_base_path = "/kaggle/input/baai-bge-base-en-v15-save/baai-bge-base-en-v15"
gte_large_path = "/kaggle/working/gte-large-en-v15-save/alibaba_combined_model"
candidate_first = 25
candidate_first_test = 15 

In [11]:
# =================================
# Retrieval Functions1
# =================================
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


task = 'Given a math problem statement and an incorrect answer as a query, retrieve relevant passages that identify and explain the nature of the error.'

In [12]:
# =================================
# 3rd stage
# =================================
tokenizer_3rd_path = "/kaggle/input/qwen-qwen2-5-32b-instruct-gptq-int4/Qwen2.5-32B-Instruct-GPTQ-Int4"
prompt_template_3rd = """
You are a Mathematics teacher. Your task is to determine if the incorrect AnswerText aligns with the provided Misconception for the given Question about {ConstructName} ({SubjectName}).

Return "Yes" if the incorrect AnswerText reflects the Misconception, otherwise return "No". Do not provide explanations.

Answer "Yes" or "No" only.
- Question: {QuestionText}
- Correct Answer: {CorrectAnswerText}
- Incorrect Answer: {AnswerText}
- Misconception: {MisconceptionName}
"""

def make_prompt_3rd(row,tokenizer,prompt_template):
    """
    Generate a prompt based on the given row and template version.
    """
    messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt_template.format(
        ConstructName=row["ConstructName"],
            AnswerText=row["AnswerText"],
            CorrectAnswerText=row["CorrectAnswerText"],
            SubjectName=row["SubjectName"],
            QuestionText=row["QuestionText"],
            MisconceptionName=row["MisconceptionName"]
    )}]
    return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            
        )

In [13]:
# # ===================
# # 2nd
# # ===================

# def make_prompt(row,prompt_template):
#     """
#     Generate a prompt based on the given row and template version.
#     """
#     return prompt_template.format(
#         ConstructName=row["ConstructName"],
#         AnswerText=row["AnswerText"],
#         SubjectName=row["SubjectName"],
#         QuestionText=row["QuestionText"],
#         MisconceptionName=row["MisconceptionName"]
#     )

# def tokenize(
#     tokenizer, prompt, max_length=max_length_2nd_stage 
#     ):
#     prompt = [i for i in prompt]
#     tokenized = tokenizer(prompt, max_length=max_length, truncation=True, padding=False)
#     input_ids = tokenized.input_ids
#     attention_mask = tokenized.attention_mask
#     return input_ids, attention_mask

# @torch.no_grad()
# @torch.cuda.amp.autocast()
# def inference(df, model, device, batch_size=batch_size_2nd_stage, max_length=max_length_2nd_stage ):
#     pred_list = []
#     for start_idx in tqdm(range(0, len(df), batch_size)):
#         end_idx = min(start_idx + batch_size, len(df))
#         tmp = df.iloc[start_idx:end_idx]
#         input_ids = tmp["input_ids"].to_list()
#         attention_mask = tmp["attention_mask"].to_list()
#         inputs = pad_without_fast_tokenizer_warning(
#             tokenizer,
#             {"input_ids": input_ids, "attention_mask": attention_mask},
#             padding="longest",
#             pad_to_multiple_of=None,
#             return_tensors="pt",
#         )
#         outputs = model(**inputs.to(device))
#         outputs.logits.detach().cpu().numpy()
#         pred_list.extend(outputs.logits.detach().cpu().numpy().reshape(-1))
    
#     df["pred"] = pred_list
    
#     return df

In [14]:
# ============================
# main
# ============================
test = pd.read_csv(TEST_PATH)
misconception = pd.read_csv(MISCONCEPTION_PATH)
if len(test) == 3:
    sample_submission = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv")
    sample_submission.to_csv("submission.csv", index=False)
    import sys
    sys.exit(0)

SystemExit: 0

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# ============================
# llm generate
# ============================
tokenizer = AutoTokenizer.from_pretrained(llm_generate_model_path)
test_pivot = []
common_cols = ['QuestionId', 'ConstructId', 'ConstructName', 'SubjectId',
               'SubjectName', 'CorrectAnswer', 'QuestionText']
for i in ["A", "B", "C", "D"]:
    test_ = test.copy()
    test_ = test_[common_cols + [f"Answer{i}Text"]]
    test_ = test_.rename({f"Answer{i}Text": "AnswerText"}, axis=1)
    test_["ans"] = i
    test_pivot.append(test_)
test_pivot = pd.concat(test_pivot).reset_index(drop=True)
test_pivot = test_pivot[test_pivot["CorrectAnswer"] != test_pivot["ans"]].reset_index(drop=True)
test_pivot["prompt"] = test_pivot.apply(
    lambda row: llm_generate_make_prompt(row), axis=1)
text_list = []
from tqdm import tqdm
for p in tqdm(test_pivot["prompt"]):
    text = tokenizer.apply_chat_template(
        p,
        tokenize=False,
        add_generation_prompt=True
    )
    text_list.append(text)
test_pivot["text"] = text_list
test_pivot = test_pivot.sort_values(by="QuestionId").reset_index(drop=True)

In [ ]:
test_pivot.to_parquet("test.parquet")

In [ ]:
%%writefile run_vllm.py

import re
import vllm
import pandas as pd
import gc
import torch
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=1028,
    disable_log_stats=True,
    enable_prefix_caching=True
)
tokenizer = llm.get_tokenizer()
train_pivot = pd.read_parquet("test.parquet")

responses = llm.generate(
    train_pivot["text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=160,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)
del llm

gc.collect()
torch.cuda.empty_cache()

responses = [x.outputs[0].text for x in responses]
train_pivot["llmMisconception"] = responses
train_pivot.to_csv("test_add_text.csv", index=False)
del responses
gc.collect()

In [ ]:
!python run_vllm.py

In [ ]:
gc.collect()

In [ ]:
# ============================
# retrieval 
# ============================

# preprocess
test_pivot = []
common_cols = ['QuestionId', 'ConstructId', 'ConstructName', 'SubjectId',
       'SubjectName', 'CorrectAnswer', 'QuestionText']
for i in ["A","B","C","D"]:
    test_ = test.copy()
    test_ = test_[common_cols + [f"Answer{i}Text"]].reset_index(drop=True)
    test_ = test_.rename({f"Answer{i}Text":"AnswerText"},axis=1)
    test_["ans"] = i
    test_pivot.append(test_)
test_pivot =  pd.concat(test_pivot).reset_index(drop=True)

# add correct answer
test_pivot_correct_ans = test_pivot[
    test_pivot["CorrectAnswer"] == test_pivot["ans"]].reset_index(drop=True)
test_pivot_correct_ans = test_pivot_correct_ans[[
    "QuestionId", "AnswerText"]].reset_index(drop=True)
test_pivot_correct_ans.columns = ["QuestionId", "CorrectAnswerText"]
test_pivot = test_pivot.merge(test_pivot_correct_ans,how="left",on="QuestionId")

# llmoutput
llm_text = pd.read_csv("test_add_text.csv")
test_pivot = test_pivot.merge(llm_text[["QuestionId","ans","llmMisconception"]],how="left",on=["QuestionId","ans"])
test_pivot = test_pivot[test_pivot["CorrectAnswer"] != test_pivot["ans"]].reset_index(drop=True)

# BAAI/bge-large-en-v1.5
# Alibaba-NLP/gte-large-en-v1.5
test_pivot["all_text"] = '<Construct> ' + test_pivot['ConstructName'] + \
                         ' <Subject> ' + test_pivot['SubjectName'] + \
                         ' <Question> ' + test_pivot['QuestionText'] + \
                         ' <Answer> ' + test_pivot['AnswerText'] + \
                         ' <LLM OUTPUT> ' +  test_pivot['llmMisconception']

# Salesforce/SFR-Embedding-2_R
test_pivot["all_text_SFR"] = ' <Question> ' + test_pivot['QuestionText'] + \
    ' <Correct Answer> ' + test_pivot['CorrectAnswerText'] + \
    ' <Incorrect Answer> ' + test_pivot['AnswerText'] + \
    ' <Construct> ' + test_pivot['ConstructName'] + \
    ' <Subject> ' + test_pivot['SubjectName'] + \
    ' <LLMOutput> ' + test_pivot['llmMisconception']

all_text_SFR_list = []
for t in test_pivot["all_text_SFR"].values:
    all_text_SFR_list.append(get_detailed_instruct(task, t))
test_pivot["all_text_SFR"] = all_text_SFR_list

In [ ]:
test_pivot["num"] = np.arange(len(test_pivot))
misconception["num"] = np.arange(len(misconception))

In [ ]:
# ================================================
# qwen14b
# ================================================
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/m/qwen-lm/qwen2.5/transformers/14b-instruct/1")
token_len = []
for t in test_pivot["all_text_SFR"]:
    token_len.append(len(tokenizer(t)["input_ids"]))
test_pivot["token_len"] = token_len
token_len = []
for t in misconception["MisconceptionName"]:
    token_len.append(len(tokenizer(t)["input_ids"]))
misconception["token_len"] = token_len

In [ ]:
test_pivot = test_pivot.sort_values(by="token_len").reset_index(drop=True)
misconception = misconception.sort_values(by="token_len").reset_index(drop=True)

In [ ]:
test_pivot1 = test_pivot.iloc[0::2].copy()
test_pivot2 = test_pivot.iloc[1::2].copy()
misconception1 = misconception.iloc[0::2].copy()
misconception2 = misconception.iloc[1::2].copy()

In [ ]:
# save
test_pivot.to_csv('test2.csv', index=False)
test_pivot1.to_csv("test2_1.csv",index=False)
test_pivot2.to_csv("test2_2.csv",index=False)
misconception.to_csv('misconception2.csv', index=False)
misconception1.to_csv("misconception2_1.csv",index=False)
misconception2.to_csv("misconception2_2.csv",index=False)

In [ ]:
test_pivot

In [ ]:
misconception

In [ ]:
%%writefile qwen_inference.py
# Qwen/Qwen2.5-14B-Instruct
# ==========================
# exp345
# ==========================
import argparse
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
#%env TOKENIZERS_PARALLELISM=true
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
from pathlib import Path
from cuml.neighbors import NearestNeighbors
import torch.nn.functional as F
import gc
from dataclasses import dataclass
from peft import PeftModel
from transformers import (
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from transformers import BitsAndBytesConfig
from concurrent.futures import ThreadPoolExecutor
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import LoraConfig, get_peft_model
from torch import nn, Tensor
import os
import gc

# ==========================
# settings
# ==========================
qwen_14b_path = "/kaggle/input/m/qwen-lm/qwen2.5/transformers/14b-instruct/1"
#test_path = "test2.parquet"
#misconception_path = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv"
device = torch.device('cuda')
class CFG:
    exp="345"
    max_len=384
    batch_size=6
    tokenizer_path=qwen_14b_path
    model_path=f"/kaggle/input/eedi-exp{exp}"
    n_fold=[0,1,2,3,4]
    lora_r=32
    lora_alpha=64


# ==============================
# Qwen/Qwen2.5-14B-Instruct
# ==============================
def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
    
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference_2nd_stage(model, sentences, misconceptions,tokenizer,
                        device, batch_size=8, max_length=384):
    sentences_emb = []
    for start_index in range(0, len(sentences), batch_size):
        sentences_batch = sentences[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model.model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().astype(np.float32)
        sentences_emb.append(embeddings)
    sentences_emb = np.concatenate(sentences_emb,axis=0)
    misconception_emb = []
    for start_index in range(0, len(misconceptions), batch_size):
        sentences_batch = misconceptions[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model.model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().astype(np.float32)
        misconception_emb.append(embeddings)
    
    misconception_emb = np.concatenate(misconception_emb,axis=0)
    return sentences_emb,misconception_emb


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('test_path')
    parser.add_argument('misconception_path')
    parser.add_argument('result_file_test')
    parser.add_argument('result_file_misconception')
    args = parser.parse_args()
    test_emb_all = []
    misconception_emb_all = []
    tokenizer = AutoTokenizer.from_pretrained(CFG.tokenizer_path)
    test_pivot = pd.read_csv(args.test_path )
    misconception = pd.read_csv(args.misconception_path)
    for fold in tqdm(CFG.n_fold):
        lora_path = CFG.model_path + f"/fold{fold}/adapter.bin"
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="fp4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
        )
        model = AutoModel.from_pretrained(CFG.tokenizer_path, 
                                          quantization_config=bnb_config,
                                          device_map=device)
        config = LoraConfig(
            r=CFG.lora_r,
            lora_alpha=CFG.lora_alpha,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
            ],
            bias="none",
            lora_dropout=0.05,  # Conventional
            task_type="CAUSAL_LM",
        )
        model = get_peft_model(model, config)
        d = torch.load(lora_path, map_location=model.device)
        model.load_state_dict(d, strict=False)
        model = model.eval()
        sentences = list(test_pivot['all_text_SFR'].values)
        misconception_list = list(misconception['MisconceptionName'].values)
        sentences_emb,misconception_emb = inference_2nd_stage(model, sentences, misconception_list,tokenizer,
                            device)
        test_emb_all.append(sentences_emb)
        misconception_emb_all.append(misconception_emb)
        del model
        torch.cuda.empty_cache()
        gc.collect()
    misconception_emb_all = np.concatenate(misconception_emb_all,axis=1)
    test_emb_all = np.concatenate(test_emb_all,axis=1)
    np.save(args.result_file_test,test_emb_all)
    np.save(args.result_file_misconception,misconception_emb_all)

In [ ]:
!(CUDA_VISIBLE_DEVICES=0 TOKENIZERS_PARALLELISM=false python qwen_inference.py test2_1.csv misconception2_1.csv qwen_test_emb_all1.npy qwen_misconception_emb_all1.npy \
 &CUDA_VISIBLE_DEVICES=1 TOKENIZERS_PARALLELISM=false python qwen_inference.py test2_2.csv misconception2_2.csv qwen_test_emb_all2.npy qwen_misconception_emb_all2.npy \
 & wait \
 )

# 32b retrieval

In [ ]:
# config 書き換え用に一旦移す
!mkdir /kaggle/temp
!cp -r /kaggle/input/m/qwen-lm/qwen2.5/transformers/32b-instruct-gptq-int4/1 /kaggle/temp/qwen2.5-32b-instruct-gptq-int4
!ls /kaggle/temp/qwen2.5-32b-instruct-gptq-int4

In [ ]:
%%writefile qwen_inference.py

import re
import vllm
import pandas as pd
import gc
import torch
import json
import numpy as np
from contextlib import ContextDecorator
import os
misconception_path = "misconception2.csv" #"/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv"

class CFG:
    exp="009"
    max_len=384
    batch_size=6
    model_path="/kaggle/temp/qwen2.5-32b-instruct-gptq-int4"
    lora_path_dir="/kaggle/input/charmq-qwen2-32b-gptq-int4-convert-loras-exp012"
    n_fold=[0,1]
    lora_r=32
    lora_alpha=64
    
print(vllm.__version__)
class ModifyJson(ContextDecorator):
    def __init__(self, json_file, changes):
        self.json_file = json_file
        self.changes = changes
        self.original_data = None

    def __enter__(self):
        with open(self.json_file, "r") as f:
            self.original_data = json.load(f)
        modified_data = self.original_data.copy()
        modified_data.update(self.changes)
        with open(self.json_file, "w") as f:
            json.dump(modified_data, f, indent=4)

    def __exit__(self, exc_type, exc_value, traceback):
        with open(self.json_file, "w") as f:
            json.dump(self.original_data, f, indent=4)


def vllm_init():
    import sys

    from vllm.model_executor.layers.pooler import Pooler, PoolingType
    from vllm.model_executor.models.qwen2 import Qwen2EmbeddingModel

    class Qwen2LastPoolEmbeddingModel(Qwen2EmbeddingModel):
        def __init__(self, *, vllm_config, prefix: str = ""):
            super().__init__(vllm_config=vllm_config, prefix=prefix)
            pooler_config = vllm_config.model_config.pooler_config
            self._pooler = Pooler.from_config_with_defaults(
                pooler_config, pooling_type=PoolingType.LAST, normalize=True, softmax=False
            )
            print("Qwen2LastPoolEmbeddingModel")

        @staticmethod
        def hello():
            print("hello")

    sys.modules["vllm"].model_executor.models.qwen2.Qwen2EmbeddingModel = Qwen2LastPoolEmbeddingModel
    from vllm.model_executor.models.qwen2 import Qwen2EmbeddingModel

@ModifyJson(
    json_file=f"{CFG.model_path}/config.json",
    changes={"architectures": ["Qwen2Model"]},
)
def inference(df):
    import vllm
    from vllm.lora.request import LoRARequest

    vllm_init()
    df_misconception_mapping = pd.read_csv(misconception_path)

    llm = vllm.LLM(
        CFG.model_path,
        tensor_parallel_size=2,
        gpu_memory_utilization=0.95,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=4096,
        disable_log_stats=True,
        enable_prefix_caching=True,
        enable_lora=True,
        max_lora_rank=CFG.lora_r,
    )

    queries = df["all_text_SFR"].to_numpy() # TODO: プロンプトを変える
    misconceptions = df_misconception_mapping["MisconceptionName"].to_numpy()

    test_emb_all_list = []
    misconception_emb_all_list = []

    for fold in CFG.n_fold:
        lora_path = os.path.join(CFG.lora_path_dir, f'fold{fold}')
        responses = llm.encode(
            np.concatenate([queries, misconceptions]),
            use_tqdm=True,
            lora_request=LoRARequest("sql_adapter", 1, lora_path),
        )
        embeddings = np.array([response.outputs.embedding for response in responses])
        test_emb_all = embeddings[: len(queries)]
        misconception_emb_all = embeddings[len(queries) :]
        test_emb_all_list.append(test_emb_all)
        misconception_emb_all_list.append(misconception_emb_all)

    test_emb_all = np.concatenate(test_emb_all_list,axis=1)
    misconception_emb_all = np.concatenate(misconception_emb_all_list,axis=1)
    return test_emb_all, misconception_emb_all


train_pivot = pd.read_csv("test2.csv")
test_emb_all, misconception_emb_all = inference(train_pivot)

np.save(f"qwen_misconception_emb_all_32b.npy",misconception_emb_all)
np.save(f"qwen_test_emb_all_32b.npy",test_emb_all)

In [ ]:
!python qwen_inference.py

In [ ]:
# ====================
# candidate
# ====================
misconception_emb_all1 = np.load("/kaggle/working/qwen_misconception_emb_all1.npy")
misconception_emb_all2 = np.load("/kaggle/working/qwen_misconception_emb_all2.npy")
test_emb_all1 = np.load("/kaggle/working/qwen_test_emb_all1.npy")
test_emb_all2 = np.load("/kaggle/working/qwen_test_emb_all2.npy")
test_pivot = pd.read_csv('test2.csv')
misconception = pd.read_csv('misconception2.csv')

test_pivot_ = pd.concat([test_pivot1,test_pivot2]).reset_index(drop=True)
misconception_ = pd.concat([misconception1,misconception2]).reset_index(drop=True)
misconception_emb_all = np.concatenate([misconception_emb_all1,misconception_emb_all2],axis=0)
test_emb_all = np.concatenate([test_emb_all1,test_emb_all2],axis=0)
del misconception_emb_all1,misconception_emb_all2,test_emb_all1,test_emb_all2
gc.collect()

test_pivot_["num2"] = np.arange(len(test_pivot_))
misconception_["num2"] = np.arange(len(misconception_))
test_pivot_ = test_pivot_.sort_values(by="num").reset_index(drop=True)
misconception_ = misconception_.sort_values(by="num").reset_index(drop=True)
misconception_emb_all1 = misconception_emb_all[misconception_["num2"].values]
test_emb_all1 = test_emb_all[test_pivot_["num2"].values]

In [ ]:
misconception_emb_all = np.load("/kaggle/working/qwen_misconception_emb_all_32b.npy")
test_emb_all = np.load("/kaggle/working/qwen_test_emb_all_32b.npy")
test_pivot = pd.read_csv('test2.csv')
misconception = pd.read_csv('misconception2.csv')

test_pivot_ = pd.read_csv('test2.csv')
misconception_ = pd.read_csv('misconception2.csv')

test_pivot_["num2"] = np.arange(len(test_pivot_))
misconception_["num2"] = np.arange(len(misconception_))
test_pivot_ = test_pivot_.sort_values(by="num").reset_index(drop=True)
misconception_ = misconception_.sort_values(by="num").reset_index(drop=True)
misconception_emb_all2 = misconception_emb_all[misconception_["num2"].values]
test_emb_all2 = test_emb_all[test_pivot_["num2"].values]


In [ ]:

misconception_emb_all = np.concatenate([misconception_emb_all1, misconception_emb_all2],axis=1)
test_emb_all = np.concatenate([test_emb_all1, test_emb_all2],axis=1)

In [ ]:
test_pivot = pd.read_csv('test2.csv')
misconception = pd.read_csv('misconception2.csv')

test_pivot = test_pivot.sort_values(by="num").reset_index(drop=True)
misconception = misconception.sort_values(by="num").reset_index(drop=True)
gc.collect()

In [ ]:
knn = NearestNeighbors(n_neighbors=candidate_first,
                       metric="cosine")
knn.fit(misconception_emb_all)
dists, nears = knn.kneighbors(test_emb_all)
#del misconception_emb_all,test_emb_all
gc.collect()

In [ ]:
# ====================
# not train
# ====================
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
misconception_ = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
misconception_list = []
for i in ["A","B","C","D"]:
    misconception_list += list(train[f"Misconception{i}Id"].unique())
misconception_list = pd.Series(misconception_list)
misconception_list = misconception_list.dropna()
misconception_list = misconception_list.astype(int).values

train_misconception_list = []
for n,m in enumerate(misconception_["MisconceptionId"]):
    if m in misconception_list:
        train_misconception_list.append(1)
    else:
        train_misconception_list.append(0)
train_misconception_list = np.array(train_misconception_list)

test_misconception = misconception_["MisconceptionId"].values
test_misconception = test_misconception[train_misconception_list == 0]

test_misconception_dict={}
for n,m in enumerate(test_misconception):
    test_misconception_dict[n] = m


misconception_emb_all_test = misconception_emb_all[train_misconception_list == 0]
knn = NearestNeighbors(n_neighbors=candidate_first_test,
                       metric="cosine")
knn.fit(misconception_emb_all_test)
_, nears_test = knn.kneighbors(test_emb_all)
del misconception_emb_all,test_emb_all,misconception_emb_all_test
gc.collect()

In [ ]:
test_pivot["QuestionId_Answer"] = test_pivot["QuestionId"].astype(str) + "_" + test_pivot["ans"].astype(str) 

In [ ]:
# sub = []
# for i,p in zip(test_pivot["QuestionId_Answer"],nears):
#     pred_list = ' '.join(map(str, p))
#     sub.append([i,pred_list])
# sub = pd.DataFrame(sub)
# sub.columns = ["QuestionId_Answer","MisconceptionId"]
# sub[["QuestionId_Answer","MisconceptionId"]].to_csv("submission.csv",index=False)

In [ ]:
# sub[["QuestionId_Answer","MisconceptionId"]]

In [ ]:
# =====================
# 3rd preprocess
# =====================
tokenizer = AutoTokenizer.from_pretrained(tokenizer_3rd_path)
candidate_pivot = []
for n,(q,a) in enumerate(zip(test_pivot["QuestionId"],test_pivot["ans"])):
    candidate = pd.DataFrame()
    candidate_ = nears[n]
    candidate["MisconceptionId"] = candidate_
    candidate["QuestionId"] = q
    candidate["ans"] = a
    candidate_pivot.append(candidate)
    
for n,(q,a) in enumerate(zip(test_pivot["QuestionId"],test_pivot["ans"])):
    candidate = pd.DataFrame()
    candidate_ = nears_test[n]
    candidate_ = [test_misconception_dict[i] for i in candidate_]
    candidate["MisconceptionId"] = candidate_
    candidate["QuestionId"] = q
    candidate["ans"] = a
    candidate_pivot.append(candidate)

candidate_pivot = pd.concat(candidate_pivot).reset_index(drop=True)
candidate_pivot = candidate_pivot.drop_duplicates(subset = ["MisconceptionId","QuestionId","ans"]).reset_index(drop=True)
merge_cols = ["QuestionId", "ConstructName",
              "SubjectName", "QuestionText", "AnswerText", "ans","CorrectAnswerText"]
candidate_pivot = candidate_pivot.merge(
    test_pivot[merge_cols], how="left", on=["QuestionId", "ans"])
candidate_pivot = candidate_pivot.merge(
    misconception, how="left", on="MisconceptionId")
candidate_pivot["prompt"] = candidate_pivot.apply(
    lambda row: make_prompt_3rd(row,tokenizer,prompt_template_3rd), axis=1)
candidate_pivot["prompt"] = candidate_pivot["prompt"] + "Answer:"

In [ ]:
candidate_pivot = candidate_pivot.sort_values(by=["QuestionId","ans"]).reset_index(drop=True)
candidate_pivot.to_csv("test3.csv",index=False)

In [ ]:
candidate_pivot["prompt"].iloc[0]

In [ ]:
%%writefile run_3rd.py

import vllm
from typing import Any, Dict, List
from transformers import LogitsProcessor
import torch
import pandas as pd
from vllm.lora.request import LoRARequest
import math
import numpy as np

def generate_text_vllm(prompts, tokenizer, model):
    sampling_params = vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=True,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        logits_processors=logits_processors,
        logprobs = 5,
    )
    
    responses = model.generate(
        prompts,
        sampling_params=sampling_params,
        use_tqdm=True,
        lora_request=LoRARequest("sql_adapter", 1,"/kaggle/input/exp341-347-lora-merge/exp341-347"),
    )

    results = []
    errors = 0    
    
    for i,response in enumerate(responses):
        try:
            x = response.outputs[0].logprobs[0]
            logprobs = []
            for k in KEEP:
                if k in x:
                    logprobs.append( math.exp(x[k].logprob) )
                else:
                    logprobs.append( 0 )
                    print(f"bad logits {i}")
            logprobs = np.array( logprobs )
            #logprobs /= logprobs.sum()
            results.append( logprobs )
        except:
            #print(f"error {i}")
            results.append( np.array([0.5]) )
            errors += 1    
    
#     response_texts = []
#     for response in responses:
#         # total_tokens += len(response.outputs[0].token_ids)
#         # response_texts.append(response.outputs[0].text)
#         response_texts.append(response.outputs[0].logprobs[0])
    return results, errors
    
df = pd.read_csv("test3.csv")
prompts = df["prompt"].values
llm = vllm.LLM(
    "/kaggle/input/qwen-qwen2-5-32b-instruct-gptq-int4/Qwen2.5-32B-Instruct-GPTQ-Int4", # "deepseek-ai/deepseek-math-7b-instruct"
    #quantization="gptq",
    tensor_parallel_size=2, # 2, 4 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=3096,
    enable_lora=True, 
    enable_prefix_caching=True,
    max_lora_rank=32
)
tokenizer = llm.get_tokenizer()

choices = ["Yes","No"]

KEEP = []
for x in choices:
    c = tokenizer.encode(x, add_special_tokens=False)[0]
    KEEP.append(c)
print(f"Force predictions to be tokens {KEEP} which are {choices}.")

class DigitLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer):
        self.allowed_ids = KEEP
        
    def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
        scores[self.allowed_ids] += 100
        return scores

logits_processors = [DigitLogitsProcessor(tokenizer)]

results, errors = generate_text_vllm(prompts, tokenizer, llm)
print(errors)
df["results"] = results
df.to_csv("test4.csv",index=False)

In [ ]:
!python run_3rd.py

In [ ]:
%%writefile run_3rd_2.py

import vllm
from typing import Any, Dict, List
from transformers import LogitsProcessor
import torch
import pandas as pd
from vllm.lora.request import LoRARequest
import math
import numpy as np

def generate_text_vllm(prompts, tokenizer, model):
    sampling_params = vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=True,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        logits_processors=logits_processors,
        logprobs = 5,
    )
    
    responses = model.generate(
        prompts,
        sampling_params=sampling_params,
        use_tqdm=True,
        lora_request=LoRARequest("sql_adapter", 1,"/kaggle/input/exp348-349-lora-merge/exp348-349"),
    )

    results = []
    errors = 0    
    
    for i,response in enumerate(responses):
        try:
            x = response.outputs[0].logprobs[0]
            logprobs = []
            for k in KEEP:
                if k in x:
                    logprobs.append( math.exp(x[k].logprob) )
                else:
                    logprobs.append( 0 )
                    print(f"bad logits {i}")
            logprobs = np.array( logprobs )
            #logprobs /= logprobs.sum()
            results.append( logprobs )
        except:
            #print(f"error {i}")
            results.append( np.array([0.5]) )
            errors += 1    
    
#     response_texts = []
#     for response in responses:
#         # total_tokens += len(response.outputs[0].token_ids)
#         # response_texts.append(response.outputs[0].text)
#         response_texts.append(response.outputs[0].logprobs[0])
    return results, errors
    
df = pd.read_csv("test3.csv")
prompts = df["prompt"].values
llm = vllm.LLM(
    "/kaggle/input/qwen-qwen2-5-32b-instruct-gptq-int4/Qwen2.5-32B-Instruct-GPTQ-Int4", # "deepseek-ai/deepseek-math-7b-instruct"
    #quantization="gptq",
    tensor_parallel_size=2, # 2, 4 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=3096,
    enable_lora=True, 
    enable_prefix_caching=True,
    max_lora_rank=32
)
tokenizer = llm.get_tokenizer()

choices = ["Yes","No"]

KEEP = []
for x in choices:
    c = tokenizer.encode(x, add_special_tokens=False)[0]
    KEEP.append(c)
print(f"Force predictions to be tokens {KEEP} which are {choices}.")

class DigitLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer):
        self.allowed_ids = KEEP
        
    def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
        scores[self.allowed_ids] += 100
        return scores

logits_processors = [DigitLogitsProcessor(tokenizer)]

results, errors = generate_text_vllm(prompts, tokenizer, llm)
print(errors)
df["results"] = results
df.to_csv("test5.csv",index=False)

In [ ]:
!python run_3rd_2.py

In [ ]:
%%writefile run_3rd_3.py

import vllm
from typing import Any, Dict, List
from transformers import LogitsProcessor
import torch
import pandas as pd
from vllm.lora.request import LoRARequest
import math
import numpy as np

def generate_text_vllm(prompts, tokenizer, model):
    sampling_params = vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=True,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        logits_processors=logits_processors,
        logprobs = 5,
    )
    
    responses = model.generate(
        prompts,
        sampling_params=sampling_params,
        use_tqdm=True,
        lora_request=LoRARequest("sql_adapter", 1,"/kaggle/input/charmq-eedi-exp015-lora-merge/charmq-exp015"),
    )

    results = []
    errors = 0    
    
    for i,response in enumerate(responses):
        try:
            x = response.outputs[0].logprobs[0]
            logprobs = []
            for k in KEEP:
                if k in x:
                    logprobs.append( math.exp(x[k].logprob) )
                else:
                    logprobs.append( 0 )
                    print(f"bad logits {i}")
            logprobs = np.array( logprobs )
            #logprobs /= logprobs.sum()
            results.append( logprobs )
        except:
            #print(f"error {i}")
            results.append( np.array([0.5]) )
            errors += 1    
    
#     response_texts = []
#     for response in responses:
#         # total_tokens += len(response.outputs[0].token_ids)
#         # response_texts.append(response.outputs[0].text)
#         response_texts.append(response.outputs[0].logprobs[0])
    return results, errors
    
df = pd.read_csv("test3.csv")
prompts = df["prompt"].values
llm = vllm.LLM(
    "/kaggle/input/qwen-qwen2-5-32b-instruct-gptq-int4/Qwen2.5-32B-Instruct-GPTQ-Int4", # "deepseek-ai/deepseek-math-7b-instruct"
    #quantization="gptq",
    tensor_parallel_size=2, # 2, 4 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=3096,
    enable_lora=True, 
    enable_prefix_caching=True,
    max_lora_rank=32
)
tokenizer = llm.get_tokenizer()

choices = ["Yes","No"]

KEEP = []
for x in choices:
    c = tokenizer.encode(x, add_special_tokens=False)[0]
    KEEP.append(c)
print(f"Force predictions to be tokens {KEEP} which are {choices}.")

class DigitLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer):
        self.allowed_ids = KEEP
        
    def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
        scores[self.allowed_ids] += 100
        return scores

logits_processors = [DigitLogitsProcessor(tokenizer)]

results, errors = generate_text_vllm(prompts, tokenizer, llm)
print(errors)
df["results"] = results
df.to_csv("test6.csv",index=False)

In [ ]:
!python run_3rd_3.py

In [ ]:
test_pred = pd.read_csv("/kaggle/working/test4.csv")
test_pred2 = pd.read_csv("/kaggle/working/test5.csv")
test_pred3 = pd.read_csv("/kaggle/working/test6.csv")

In [ ]:
pred = []
results = test_pred["results"]
for r in results:
    r = float(r.split(" ")[0][1:])
    pred.append(r)
test_pred["pred"] = pred

pred = []
results = test_pred2["results"]
for r in results:
    r = float(r.split(" ")[0][1:])
    pred.append(r)
test_pred2["pred2"] = pred

pred = []
results = test_pred3["results"]
for r in results:
    r = float(r.split(" ")[0][1:])
    pred.append(r)
test_pred3["pred3"] = pred

test_pred["pred"] = (test_pred["pred"] + test_pred2["pred2"] + 4*test_pred3["pred3"])/6

In [ ]:
train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv")
misconception_list = []
for i in ["A","B","C","D"]:
    misconception_list += list(train[f"Misconception{i}Id"].unique())
misconception_list = pd.Series(misconception_list)
misconception_list = misconception_list.dropna()
misconception_list = misconception_list.astype(int).values
test_pred.loc[test_pred["MisconceptionId"].isin(misconception_list),
"pred"] = test_pred.loc[test_pred["MisconceptionId"].isin(misconception_list),"pred"]*0.0

In [ ]:
test_pred["id"] = test_pred["QuestionId"].astype(str) + "_" + test_pred["ans"].astype(str) 
test_pred["pred_rank"] = test_pred.groupby(by="id")["pred"].rank(ascending=False)

In [ ]:
test_pred = test_pred.sort_values(by=["id","pred_rank"]).reset_index(drop=True)
group = test_pred.groupby(by="id")
sub = []
for g,df in group:
    df = df.sort_values(by="pred_rank").reset_index(drop=True)
    pred_ = df["MisconceptionId"].iloc[:25].values
    pred_list = ' '.join(map(str, pred_))
    sub.append([g,pred_list])
sub = pd.DataFrame(sub)
sub.columns = ["QuestionId_Answer","MisconceptionId"]
sub[["QuestionId_Answer","MisconceptionId"]].to_csv("submission.csv",index=False)

In [ ]:
sub